In [29]:
import numpy
import pandas
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import itertools 

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from datetime import datetime,timedelta

from xgboost import XGBClassifier

In [31]:
news_data = pd.read_csv('news_categ.csv')
news_data = news_data.iloc[:,1:]
news_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.000035,0.000314,0.000140,0.000070,0.000070,0.000140,0.000140,0.000210,0.000140,0.000140,0.000210,0.000105,0.000105,0.000000,0.000140,0.000000,0.000559,2019-06-01
1,0.000070,0.000070,0.000140,0.000175,0.000035,0.000070,0.000314,0.000175,0.000175,0.000000,0.000210,0.000070,0.000105,0.000070,0.000175,0.000210,0.000140,2020-01-06
2,0.000070,0.000175,0.000105,0.000070,0.000000,0.000105,0.000070,0.000070,0.000035,0.000035,0.000105,0.000105,0.000070,0.000070,0.000140,0.000210,0.000210,2019-12-29
3,0.000035,0.000140,0.000000,0.000070,0.000210,0.000035,0.000070,0.000000,0.000140,0.000140,0.000105,0.000000,0.000244,0.000035,0.000140,0.000105,0.000175,2019-12-15
4,0.000140,0.000035,0.000244,0.000070,0.000105,0.000105,0.000140,0.000140,0.000105,0.000244,0.000105,0.000105,0.000175,0.000070,0.000035,0.000070,0.000244,2019-08-13


In [32]:
stock_data = pd.read_excel('Stock_data.xlsx')
stock_data.head()

,Dates,PX_HIGH,PX_CLOSE_1D,PX_VOLUME,PX_LOW,PX_OPEN,Unnamed: 6
0,2010-01-04,5238.45,5201.05,148652400,5167.10,5200.90,5200.00
1,2010-01-05,5288.35,5232.20,240844400,5242.40,5277.15,5200.90
2,2010-01-06,5310.85,5277.90,216147800,5260.05,5278.15,5277.15
3,2010-01-07,5302.55,5281.80,181246700,5244.75,5281.80,5278.15
4,2010-01-08,5276.75,5263.10,201910800,5234.70,5264.25,5281.80


In [33]:
#stock_data = stock_data.iloc[:,[0,1,2,5]]

### Ensuring the formatting of Date column is same for both Dataframes

In [34]:
for i in range(news_data.shape[0]):
    news_data.iloc[i,-1] = datetime.strptime(news_data.iloc[i,-1], '%Y-%m-%d').strftime('%m/%d/%y')

In [35]:
for i in range(stock_data.shape[0]):
    stock_data.iloc[i,0] = datetime.strptime(str(stock_data.iloc[i,0]), '%Y-%m-%d %H:%M:%S').strftime('%m/%d/%y')

### Creating the final dataset

In [36]:
df = []
Y = []
for i in range(stock_data.shape[0]-1):
    for j in range(news_data.shape[0]):
        
        if(news_data.iloc[j,-1]==stock_data.iloc[i,0]):
            
            #print(news_data.iloc[j,-1])
            
            z = list(stock_data.iloc[i,:]) + list(news_data.iloc[j,:-1])
            
            #Y.append(stock_data.iloc[i+1,2])
            
            #Y.append(1 if (stock_data.iloc[i+1,2] >= stock_data.iloc[i,2]) else 0)
            
            Y.append(1 if (stock_data.iloc[i+1,2] >= stock_data.iloc[i,2]) else 0)
            
            df.append(z)
        

In [37]:
df = pd.DataFrame(df)
df.shape

(253, 24)

In [38]:
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,01/01/19,10923.60,10862.55,159404542,10807.10,10881.70,10913.20,0.000070,0.000140,0.000140,...,0.000244,0.000210,0.000070,0.000175,0.000070,0.000175,0.000000,0.000070,0.000175,0.000140
1,01/02/19,10895.35,10910.10,309665939,10735.05,10868.85,10881.70,0.000175,0.000105,0.000175,...,0.000244,0.000175,0.000000,0.000244,0.000070,0.000140,0.000070,0.000070,0.000279,0.000140
2,01/03/19,10814.05,10792.50,286241745,10661.25,10796.80,10868.85,0.000140,0.000140,0.000140,...,0.000035,0.000035,0.000070,0.000349,0.000210,0.000384,0.000070,0.000175,0.000314,0.000175
3,01/04/19,10741.05,10672.25,296596655,10628.65,10699.70,10796.80,0.000175,0.000070,0.000140,...,0.000210,0.000140,0.000070,0.000140,0.000279,0.000210,0.000070,0.000105,0.000244,0.000105
4,01/07/19,10835.95,10727.35,269371080,10750.15,10804.85,10699.70,0.000105,0.000035,0.000279,...,0.000244,0.000105,0.000035,0.000105,0.000279,0.000105,0.000105,0.000035,0.000175,0.000244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,01/30/20,12150.30,12129.50,538138743,12010.60,12147.75,12114.90,0.000105,0.000000,0.000140,...,0.000314,0.000070,0.000070,0.000419,0.000105,0.000105,0.000175,0.000035,0.000175,0.000140
249,01/31/20,12103.55,12035.80,771278258,11945.85,12100.40,12147.75,0.000175,0.000035,0.000210,...,0.000175,0.000035,0.000140,0.000594,0.000140,0.000140,0.000035,0.000035,0.000105,0.000175
250,02/03/20,11749.85,11661.85,669815788,11614.50,11627.45,12100.40,0.000000,0.000175,0.000384,...,0.000698,0.000000,0.000000,0.000419,0.000070,0.000210,0.000105,0.000000,0.000105,0.000070
251,02/04/20,11986.15,11707.90,560430291,11783.40,11786.25,11627.45,0.000105,0.000035,0.000524,...,0.000244,0.000000,0.000210,0.000210,0.000035,0.000070,0.000035,0.000035,0.000140,0.000035


In [39]:
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,2019-01-01,10923.60,10862.55,159404542,10807.10,10881.70,10913.20,0.000070,0.000140,0.000140,...,0.000244,0.000210,0.000070,0.000175,0.000070,0.000175,0.000000,0.000070,0.000175,0.000140
1,2019-01-02,10895.35,10910.10,309665939,10735.05,10868.85,10881.70,0.000175,0.000105,0.000175,...,0.000244,0.000175,0.000000,0.000244,0.000070,0.000140,0.000070,0.000070,0.000279,0.000140
2,2019-01-03,10814.05,10792.50,286241745,10661.25,10796.80,10868.85,0.000140,0.000140,0.000140,...,0.000035,0.000035,0.000070,0.000349,0.000210,0.000384,0.000070,0.000175,0.000314,0.000175
3,2019-01-04,10741.05,10672.25,296596655,10628.65,10699.70,10796.80,0.000175,0.000070,0.000140,...,0.000210,0.000140,0.000070,0.000140,0.000279,0.000210,0.000070,0.000105,0.000244,0.000105
4,2019-01-07,10835.95,10727.35,269371080,10750.15,10804.85,10699.70,0.000105,0.000035,0.000279,...,0.000244,0.000105,0.000035,0.000105,0.000279,0.000105,0.000105,0.000035,0.000175,0.000244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2020-01-30,12150.30,12129.50,538138743,12010.60,12147.75,12114.90,0.000105,0.000000,0.000140,...,0.000314,0.000070,0.000070,0.000419,0.000105,0.000105,0.000175,0.000035,0.000175,0.000140
249,2020-01-31,12103.55,12035.80,771278258,11945.85,12100.40,12147.75,0.000175,0.000035,0.000210,...,0.000175,0.000035,0.000140,0.000594,0.000140,0.000140,0.000035,0.000035,0.000105,0.000175
250,2020-02-03,11749.85,11661.85,669815788,11614.50,11627.45,12100.40,0.000000,0.000175,0.000384,...,0.000698,0.000000,0.000000,0.000419,0.000070,0.000210,0.000105,0.000000,0.000105,0.000070
251,2020-02-04,11986.15,11707.90,560430291,11783.40,11786.25,11627.45,0.000105,0.000035,0.000524,...,0.000244,0.000000,0.000210,0.000210,0.000035,0.000070,0.000035,0.000035,0.000140,0.000035


In [40]:
#df['Day'] = pd.DatetimeIndex(df.iloc[:,0]).day % 7
#df['Month'] = pd.DatetimeIndex(df.iloc[:,0]).month
#df['Year'] = pd.DatetimeIndex(df.iloc[:,0]).year

In [41]:
pd.DataFrame(Y).head(5)

,0
0,1
1,0
2,0
3,1
4,1


In [42]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

encoded_Y = np_utils.to_categorical(encoded_Y)

In [43]:
from sklearn import preprocessing

from sklearn.cross_validation import train_test_split

In [44]:
#min_max_scaler = preprocessing.MinMaxScaler()
#X = min_max_scaler.fit_transform(X)
#Y = min_max_scaler.fit_transform(Y)

In [45]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], encoded_Y, test_size=0.2)

######### Normalizing the data

mean = X_train.mean(axis=0)
X_train -= mean
std = X_train.std(axis=0)
X_train /= std


mean = X_test.mean(axis=0)
X_test -= mean
std = X_test.std(axis=0)
X_test /= std

In [46]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    
    model.add(Dense(5, input_dim=23, kernel_initializer='normal', activation='relu'))
    
    model.add(Dense(5, kernel_initializer='normal', activation='relu'))
    #model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))

    model.add(Dense(2, kernel_initializer='normal', activation='softmax'))
    
    #model.summary()
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # for binayr classification
        #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  # for multi class
    return model

In [47]:
model=create_baseline();
history=model.fit(X_train, y_train, batch_size=50, epochs=50, validation_split = 0.2, verbose=1)

Train on 161 samples, validate on 41 samples
Epoch 1/50
161/161 [==============================] - 3s 17ms/step - loss: 0.6931 - acc: 0.5093 - val_loss: 0.6926 - val_acc: 0.5854
Epoch 2/50
161/161 [==============================] - 0s 96us/step - loss: 0.6926 - acc: 0.5590 - val_loss: 0.6921 - val_acc: 0.5854
Epoch 3/50
161/161 [==============================] - 0s 112us/step - loss: 0.6923 - acc: 0.5590 - val_loss: 0.6916 - val_acc: 0.5854
Epoch 4/50
161/161 [==============================] - 0s 96us/step - loss: 0.6920 - acc: 0.5590 - val_loss: 0.6912 - val_acc: 0.5854
Epoch 5/50
161/161 [==============================] - 0s 124us/step - loss: 0.6916 - acc: 0.5590 - val_loss: 0.6907 - val_acc: 0.5854
Epoch 6/50
161/161 [==============================] - 0s 118us/step - loss: 0.6913 - acc: 0.5590 - val_loss: 0.6903 - val_acc: 0.5854
Epoch 7/50
161/161 [==============================] - 0s 149us/step - loss: 0.6909 - acc: 0.5590 - val_loss: 0.6900 - val_acc: 0.5854
Epoch 8/50
161/161 [

In [48]:
#model.evaluate(X_test, y_test)
y_pred = model.predict(X_test, verbose=1)

51/51 [==============================] - 1s 16ms/step


In [55]:
def normalize(df):
    mean = df.mean(axis=0)
    df -= mean
    std = df.std(axis=0)
    df /= std
    
    return df

In [58]:
#Defining Input(X) and Output(Y)
X = np.array(df.iloc[:,1:])

Y = np.array(Y)
Y = encoded_Y

acc = []

#Iterating over number of samples

for j in range(240,len(X),10):
    
    n_train_hours = j
    
    
    s=0
    z = 0

    for i in range(len(X) - n_train_hours - 1):

        print(i+ n_train_hours + 1,"***********************************************************************")

        train_X = X[i:(i+ n_train_hours + 1), :]
        train_y = Y[i:(i+ n_train_hours + 1), :]
        test_X = X[(i+ n_train_hours + 1):(i+ n_train_hours + 3), :]
        test_y = Y[(i+ n_train_hours + 1):(i+ n_train_hours + 3), :]
        
        print(np.array(train_X))
        


        ######### Normalizing the data

        train_X = normalize(train_X)
        print(np.array(train_X))

        test_X = normalize(test_X)

        #model=create_baseline();
        #history=model.fit(train_X, train_y, batch_size=20, epochs=10, validation_split = 0.2, verbose=1)
        #y_pred = model.predict(X_test, verbose=1)
        
        z += 1

        if(sum((y_pred>=0.5).astype(int)[0]==test_y.astype(int)[0])==2):
            s += 1


    print(s)

    print(z)

    acc.append((s/z)*100)

241 ***********************************************************************
[[1.09236000e+04 1.08625500e+04 1.59404542e+08 ... 6.98397198e-05
  1.74599292e-04 1.39679440e-04]
 [1.08953500e+04 1.09101000e+04 3.09665939e+08 ... 6.98397198e-05
  2.79358879e-04 1.39679440e-04]
 [1.08140500e+04 1.07925000e+04 2.86241745e+08 ... 1.74599292e-04
  3.14278732e-04 1.74599292e-04]
 ...
 [1.22471000e+04 1.22558500e+04 4.26931711e+08 ... 3.49198599e-05
  1.39679440e-04 3.49198599e-05]
 [1.22222000e+04 1.21684500e+04 3.04078039e+08 ... 6.98397198e-05
  3.49198584e-04 6.98397198e-05]
 [1.22899000e+04 1.21825000e+04 4.07697594e+08 ... 1.04759580e-04
  1.04759580e-04 2.09519159e-04]]
[[-1.361879   -1.32978035 -1.63535489 ... -0.46541715 -0.0644033
  -0.43596991]
 [-1.42546749 -1.22419402 -0.90319942 ... -0.46541715  1.04425258
  -0.43596991]
 [-1.60846729 -1.48532868 -1.01733486 ...  0.96041582  1.41380444
  -0.19331695]
 ...
 [ 1.61721365  1.76408801 -0.33181662 ... -0.94069484 -0.43395516
  -1.163928

C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
C:\Users\prash\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in greater_equal


In [22]:
#Defining Input(X) and Output(Y)
X = np.array(df.iloc[:,1:7])

Y = np.array(Y)
Y = encoded_Y

acc1 = []

#Iterating over number of samples

for j in range(240,len(X),240):
    
    n_train_hours = j
    
    
    s=0
    z = 0

    for i in range(len(X) - n_train_hours - 1):

        print(i,"***********************************************************************")

        train_X = X[i:(i+ n_train_hours + 1), :]
        train_y = Y[i:(i+ n_train_hours + 1), :]
        test_X = X[(i+ n_train_hours + 1):(i+ n_train_hours + 3), :]
        test_y = Y[(i+ n_train_hours + 1):(i+ n_train_hours + 3), :]
        
        #print(np.array(test_y[0]))


        ######### Normalizing the data

        mean = train_X.mean(axis=0)
        train_X -= mean
        std = train_X.std(axis=0)
        train_X /= std


        mean = test_X.mean(axis=0)
        test_X -= mean
        std = test_X.std(axis=0)
        test_X /= std

        model=create_baseline();
        history=model.fit(train_X, train_y, batch_size=20, epochs=10, validation_split = 0.2, verbose=1)
        y_pred = model.predict(X_test, verbose=1)
        
        z += 1

        if(sum((y_pred>=0.5).astype(int)[0]==test_y.astype(int)[0])==2):
            s += 1


    #print(s)

    #print(z)

    acc1.append((s/z)*100)

0 ***********************************************************************


ValueError: Error when checking input: expected dense_40_input to have shape (23,) but got array with shape (6,)

In [52]:
acc

[]

In [ ]:
acc1

In [ ]:
plt.plot(range(240,len(X),50),acc1,c='blue')
plt.plot(range(240,len(X),50),acc,c='red')